## A conversation with Andrew Ng
단어들의 순서를 고려하기 위해서 RNN, GIO, LSTM과 같은 모델들을 사용함
> for. **자연어 처리**  

## Introduction
* f(data, labels) = Rules  
  * input : data, labels
  * output : rules  

> 이러한 방식은 단어들의 순서를 고려하지 않은 것!
  
* f(x0) = y0 --> f(x1) = y1 ...  

> 이러한 방식처럼 이전 결과가 다음 입력에도 영향끼침! (가중치 갱신)    
  
> **"RNN"** : Recurrent Neural Network!

## LSTMs
* **LSTM : an update to RNN (Long Short-Term Memory)**
> LSTM : RNN의 일종
* Cell states : pipeline of contexts (which LSTMs have)
> * 방향, 양방향 모두 가능 (나중 문맥이 이전 문맥에 영향줄 수 있음)
> * **Bidirectional!** (<-> feed forward)

## Implementing LSTMs in code

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # 64 : the number of outputs of this layer
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                            
])

# tf.keras.layers.LSTM(64) ==> 64 : the number of outputs of this layer
# 하지만 Output Shape에는 128! ==> 양방향이므로 2배로 늘림
# 이 LSTM층을 tf.keras.layers.Bidirectional()로 감싸면 cell state를 양방향으로 만듦

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)), # 다음 LSTM에 먹일때 인자 추가!
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), 
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')                            
])

# return_sequences = True : 이 LSTM 층의 output과 다음 LSTM 층의 input을 맞춤(match)
# 즉, 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값(hidden state)을 출력 (중간 출력들을 모두 사용!)

## Accuracy and loss
1 layer LSTM과 2 layer LSTM의 accuracy 비교
* epoch에 따른 acc 변화 양상 곡선의 부드러움 정도가 다름! (두 층일 때 더 자연스럽게 변함)
* 10 epochs -> 50 epochs로 바꾸면 더 들쭉날쭉 정도가 크게 차이남
* 결과적으로 마지막엔 acc가 더 좋을지 몰라도, 모델의 전체적인 정확성은 떨어질 수 있음 (신뢰도)

## Looking into the code
* without LSTM (with flattening, pooling) 
  * 85% accuracy
  * loss와 val_loss 모두 점점 평평해짐
* with LSTM (with Bidirectional) 
  * 97.5% accuracy
  * loss는 점점 평평해지며 줄어들지만, val_loss는 점점 늘어남..?!
  > **overfitting!**  

  
*따라서 training parameter들을 조정하는 것이 중요하다!!*


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

## Using a convolutional network
다양한 층으로 모델 구성하여 성능 비교해보기
1. flattening & pooling 층 이용
2. Bidirectional(LSTM) 층 이용
3. **convolution 층 이용** ==> 이번에 비교해 볼 것 
> 자연어 처리에서는 Conv2D가 아니라 **Conv1D** 이용

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(128, 5, activation='relu'), # generate 128 filters, filter size : 5
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

# 즉, 5 단어마다 128개의 필터들 존재! (한번 컨볼루션 5개 단어로 할때마다 128종류의 필터들 적용 => 여러 특징맵 생성)
# return : 1d array
# filter size가 5이므로, 컨볼루션 수행 시 문장 앞뒤로 단어 2개씩 잘라내게 됨! (문장 길이 : 120 --> 116)

## Going back to the IMDB dataset
1. **Flatten()**층 사용 시 : 171533개의 매개변수
  * 단, 1 epoch 당 5초
2. Bidirectional()층 (**LSTM**) 사용 시 : 30129개의 매개변수
  * 단, 1 epoch 당 43초
  * accuracy도 더 낫지만, overfitting 발생
3.  Bidirectional()층 (**GRU**) 사용 시 : 169997개의 매개변수
  * 단, 1 epoch 당 20초
  * accuracy도 좋고, 약간의 overfitting 존재

4. **Conv1D()**층 사용 시 : 171149개의 매개변수 
* 단, 1 epoch 당 6초
* accuracy : 100%에 가깝고, overfitting 발생


> cf. GRU : different type of RNN

In [ ]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

# Model Definition with LSTM
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

## Tips from Laurance
* 각각 다른 층이나 다른 옵션들로 결과 비교해보기
* overfitting을 방지하는 방법 생각해보기